In [1]:
# Installations and Imports

import numpy as np
import pandas as pd


In [2]:
#Filtering dataset
earnings_filepath = '/kaggle/input/us-historical-stock-prices-with-earnings-data/stocks_latest/earnings_latest.csv'
prices_filepath = '/kaggle/input/us-historical-stock-prices-with-earnings-data/stocks_latest/stock_prices_latest.csv'

earnings_df = pd.read_csv(earnings_filepath, usecols=['symbol', 'date'])
prices_df = pd.read_csv(prices_filepath, usecols=['symbol', 'date', 'close', 'volume'])

earnings_df = earnings_df[(earnings_df['symbol']=='AMD') | (earnings_df['symbol']=='QCOM')]

prices_df = prices_df[(prices_df['symbol']=='AMD') | (prices_df['symbol']=='QCOM')]

earnings_df['date'] = pd.to_datetime(earnings_df['date'])
earnings_df = earnings_df[earnings_df['date'].dt.year == 2018]

prices_df['date'] = pd.to_datetime(prices_df['date'])
prices_df = prices_df[prices_df['date'].dt.year == 2018]

eRow, eCol = earnings_df.shape
pRow, pCol = prices_df.shape

print(f"There are {eRow} rows and {eCol} columns in Earnings Dataframe")
print(f"There are {pRow} rows and {pCol} columns in Prices Dataframe")


There are 8 rows and 2 columns in Earnings Dataframe
There are 502 rows and 4 columns in Prices Dataframe


In [3]:
ern_AMD = sorted(earnings_df[earnings_df['symbol'] == 'AMD']['date'])
ern_QCOM = sorted(earnings_df[earnings_df['symbol'] == 'QCOM']['date'])


In [4]:
AMD_df = prices_df[(prices_df['symbol'] == 'AMD')].sort_values(by='date')
QCOM_df = prices_df[(prices_df['symbol'] == 'QCOM')].sort_values(by='date')

In [5]:
p_AMD = AMD_df['close'].tolist()
vol_AMD = AMD_df['volume'].tolist()

p_QCOM = QCOM_df['close'].tolist()
vol_QCOM = QCOM_df['volume'].tolist()

In [6]:
AMD_ern_indexes = []
QCOM_ern_indexes = []
counter = 0
for i, row in AMD_df.iterrows():
    if row['date'] in ern_AMD:
        AMD_ern_indexes.append(counter)
    counter += 1

counter = 0
for i, row in QCOM_df.iterrows():
    if row['date'] in ern_QCOM:
        QCOM_ern_indexes.append(counter)
    counter += 1
    
AMD_ern_indexes

[19, 78, 141, 205]

In [7]:
import math

no_ern_vol = vol_AMD.copy()
ern_vol = []
for i in range(len(AMD_ern_indexes)-1, -1, -1):
    idx = AMD_ern_indexes[i]
    cur_vol = no_ern_vol.pop(idx)
    ern_vol.insert(0, cur_vol)
   
avg_vol = sum(vol_AMD)/len(vol_AMD)
no_ern_avg_vol = sum(no_ern_vol)/len(no_ern_vol)
ern_avg_vol = sum(ern_vol)/len(ern_vol)

ratio = ern_avg_vol/no_ern_avg_vol

cum_diff = 0
for x in vol_AMD:
    cum_diff += (x-avg_vol)*(x-avg_vol)
    
std_dev = math.sqrt(cum_diff/len(vol_AMD))

z_scores = []
for vol in ern_vol:
    cur_z = (vol-avg_vol)/std_dev
    z_scores.append(cur_z)

print(z_scores)

[-0.595215496767348, 0.1435201863040476, -0.0054714864530792775, 1.0864593493864714]


In [8]:
print(ratio)

1.088867196321877


In [9]:
avg_vol_QCOM = sum(vol_QCOM)/len(vol_QCOM)


cumul_diff = 0
cumul_AMD = 0
cumul_QCOM = 0

for i in range(len(vol_QCOM)):
    AMD_diff = vol_AMD[i] - avg_vol
    QCOM_diff = vol_QCOM[i] - avg_vol_QCOM
    cumul_diff += AMD_diff*QCOM_diff

    cumul_AMD += (vol_AMD[i] - avg_vol)*(vol_AMD[i] - avg_vol)
    cumul_QCOM += (vol_QCOM[i] - avg_vol_QCOM)*(vol_QCOM[i] - avg_vol_QCOM)

denominator = math.sqrt(cumul_AMD*cumul_QCOM)    
inclusive_r = cumul_diff/denominator
print(inclusive_r)

0.3596177850657947


In [10]:
QCOM_no_ern_vols = vol_QCOM.copy()

for i in QCOM_ern_indexes:
    del QCOM_no_ern_vols[i]
   

QCOM_no_ern_avg = sum(QCOM_no_ern_vols)/len(QCOM_no_ern_vols)

cumul_diff = 0
cumul_AMD = 0
cumul_QCOM = 0

for i in range(len(QCOM_no_ern_vols)):
    AMD_diff = no_ern_vol[i] - no_ern_avg_vol
    QCOM_diff = QCOM_no_ern_vols[i] - QCOM_no_ern_avg
    cumul_diff += AMD_diff*QCOM_diff

    cumul_AMD += (no_ern_vol[i] - no_ern_avg_vol)*(no_ern_vol[i] - no_ern_avg_vol)
    cumul_QCOM += (QCOM_no_ern_vols[i] - QCOM_no_ern_avg)*(QCOM_no_ern_vols[i] - QCOM_no_ern_avg)

denominator = math.sqrt(cumul_AMD*cumul_QCOM)    
exclusive_r = cumul_diff/denominator
print(exclusive_r)


0.34156437646523197


In [11]:
def algo_trade(prices_A, prices_B):
    num_days = len(prices_A)
    cash = 100
    num_shares = 0
    for i in range(1, num_days):
        cur_price = prices_A[i]
        if prices_B[i] > prices_B[i-1]:
            if cash >= cur_price:                
                to_buy = int(cash/cur_price)
                num_shares += to_buy
                cash -= to_buy*cur_price
        else:
            cash += num_shares*cur_price
            num_shares = 0
            
    cash += num_shares*prices_A[-1]
    return cash

algo_trade(p_AMD, p_QCOM)

208.4873000000001

In [12]:
def buy_hold(prices_A):
    num_days = len(prices_A)
    cash = 100
    num_shares = 0
    
    to_buy = int(cash/prices_A[0])
    num_shares += to_buy
    cash -= to_buy*prices_A[0]
    
    for i in range(1, num_days):
        if cash >= prices_A[i]:             
            to_buy = int(cash/prices_A[i])
            num_shares += to_buy
            cash -= to_buy*prices_A[i]
            
    cash += num_shares*prices_A[-1]
    return cash

buy_hold(p_AMD)

167.32

In [13]:
def algo_trade_cost(prices_A, prices_B):
    num_days = len(prices_A)
    cash = 100
    num_shares = 0
    for i in range(1, num_days):
        cur_price = prices_A[i]
        if prices_B[i] > prices_B[i-1]:
            if cash >= cur_price:                
                to_buy = int(cash/cur_price)
                num_shares += to_buy
                cash -= to_buy*cur_price
        else:
            cash += 0.9975*num_shares*cur_price
            num_shares = 0
            
    cash += 0.9975*num_shares*prices_A[-1]
    return cash

algo_trade_cost(p_AMD, p_QCOM)

191.4975530000005

In [16]:
from sklearn.linear_model import LinearRegression

v_train_QCOM = vol_QCOM.copy()
p_train_QCOM = p_QCOM.copy()
v_train_AMD = vol_AMD.copy()
p_train_AMD = p_AMD.copy()

train_len = len(v_train_QCOM)
start_index = train_len - 19

v_test_QCOM = []
v_test_AMD = []
p_test_QCOM = []
p_test_AMD = []

test_inputs = np.zeros((19, 5))
test_outputs = np.zeros(19)

for i in range(start_index, train_len):
    test_idx = i-start_index
    cur_QCOM_vol = v_train_QCOM.pop(start_index)
    v_test_QCOM.append(cur_QCOM_vol)
    test_inputs[test_idx][0] = cur_QCOM_vol
    
    cur_AMD_vol = v_train_AMD.pop(start_index)
    v_test_AMD.append(cur_AMD_vol)
    test_outputs[test_idx] = cur_AMD_vol
    
    cur_QCOM_p = p_train_QCOM.pop(start_index)
    p_test_QCOM.append(cur_QCOM_p)
    test_inputs[test_idx][1] = cur_QCOM_p
    
    cur_AMD_p = p_train_AMD.pop(start_index)
    p_test_AMD.append(cur_AMD_p)
    test_inputs[test_idx][2] = cur_AMD_p
    
    if i in AMD_ern_indexes:
        test_inputs[test_idx][3] = 1
    else:
        test_inputs[test_idx][3] = 0
        
    if i in QCOM_ern_indexes:
        test_inputs[test_idx][4] = 1
    else:
        test_inputs[test_idx][4] = 0
    
train_inputs = np.zeros((start_index, 5))
train_outputs = np.zeros(start_index)

for i in range(0, start_index):
    cur_QCOM_vol = v_train_QCOM.pop(0)
    train_inputs[i][0] = cur_QCOM_vol
    
    cur_AMD_vol = v_train_AMD.pop(0)
    train_outputs[i] = cur_AMD_vol
    
    cur_QCOM_p = p_train_QCOM.pop(0)
    train_inputs[i][1] = cur_QCOM_p
    
    cur_AMD_p = p_train_AMD.pop(0)
    train_inputs[i][2] = cur_AMD_p
    
    if i in AMD_ern_indexes:
        train_inputs[i][3] = 1
    else:
        train_inputs[i][3] = 0
        
    if i in QCOM_ern_indexes:
        train_inputs[i][4] = 1
    else:
        train_inputs[i][4] = 0
        



In [15]:
sklearn_model = LinearRegression().fit(train_inputs, train_outputs)

In [17]:
from sklearn.metrics import mean_squared_error
sklearn_predictions = sklearn_model.predict(test_inputs)
mean_squared_error(sklearn_predictions, test_outputs)

348989735280726.94

In [24]:
print(sklearn_predictions[6])
print(test_outputs[6])

93921405.08293486
100340657.0


In [33]:
import statsmodels.api as sm
x = sm.add_constant(train_inputs.copy())
sm_model = sm.OLS(train_outputs, x)
results = sm_model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.487
Model:                            OLS   Adj. R-squared:                  0.476
Method:                 Least Squares   F-statistic:                     42.96
Date:                Thu, 05 Oct 2023   Prob (F-statistic):           5.22e-31
Time:                        04:22:37   Log-Likelihood:                -4353.8
No. Observations:                 232   AIC:                             8720.
Df Residuals:                     226   BIC:                             8740.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.406e+07   2.64e+07     -2.045      0.0

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

look_back = 15

train_generator = TimeseriesGenerator(v_train_AMD, v_train_AMD, length=len(v_train_AMD), batch_size=20)     
test_generator = TimeseriesGenerator(v_test_AMD, v_test_AMD, length=len(v_test_AMD), batch_size=1)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

rnn_model = Sequential()
rnn_model.add(
    LSTM(10,
        activation='relu')
)
rnn_model.add(Dense(1))
rnn_model.compile(optimizer='adam', loss='mse')

num_epochs = 25
rnn_model.fit_generator(v_train_AMD, train_outputs, epochs=num_epochs, verbose=1)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming you have the following data structures:
# train_inputs (shape: (232, 5))
# train_outputs (shape: (232,))
# test_inputs (shape: (test_len, 5))
# test_outputs (shape: (test_len,))

# Train ARIMA Model
arima_model = ARIMA(train_outputs, order=(p, d, q), exog=train_inputs)  # Replace p, d, q with appropriate values
arima_fit = arima_model.fit(disp=0)
arima_forecast, stderr, conf_int = arima_fit.forecast(steps=len(test_inputs))
arima_mse = mean_squared_error(test_outputs, arima_forecast)

# Train SARIMA Model (if applicable)
# sarima_model = SARIMAX(train_outputs, order=(p, d, q), seasonal_order=(P, D, Q, S), exog=train_inputs)  # Replace with appropriate values
# sarima_fit = sarima_model.fit(disp=0)
# sarima_forecast, stderr, conf_int = sarima_fit.forecast(steps=len(test_inputs))
# sarima_mse = mean_squared_error(test_outputs, sarima_forecast)

# Train LSTM RNN Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

input_shape = (train_inputs.shape[1], 1)  # Adjust for your input data shape
lstm_model = create_lstm_model(input_shape)
lstm_model.fit(train_inputs, train_outputs, epochs=epochs, batch_size=batch_size, verbose=0)
lstm_forecast = lstm_model.predict(test_inputs).flatten()
lstm_mse = mean_squared_error(test_outputs, lstm_forecast)

# Compare Model MSE
model_results = {
    'ARIMA': arima_mse,
    # 'SARIMA': sarima_mse,  # Uncomment if using SARIMA
    'LSTM': lstm_mse,
}

best_model = min(model_results, key=model_results.get)
print(f"Best Model: {best_model}")
print(f"MSE of {best_model}: {model_results[best_model]}")
